In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib agg

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np

import re

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader


In [ ]:
def latex_float(number):
    float_str = "{0:.3g}".format(number)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0:.2f}\times10^{{{1}}}".format(float(base), int(exponent))
    else:
        return float_str

In [ ]:
# service options
delimiter = '/'

## configuration options
data_path = '/media/nfs/PiCoPiC/m50s1/'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

timestamp=2e-9
cmap='gray_r'
image_interpolation = 'nearest'

autoselect = True
show_grid=False
aspect='equal'

guess_number_ticks=20

# cmp = 'E/r'
# specie = None # simply init with none. Replace with required specie for specied grids
cmp = 'density'
specie = 'ions'

probe_path = None

if specie:
    probe_path = cmp + delimiter + specie
else:
    probe_path = cmp

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
# size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]
size=[0, 0, 300, 1229]

r_scale = reader.meta.geometry_size[0] / reader.meta.geometry_grid[0]
z_scale = reader.meta.geometry_size[1] / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
# clim = [-clim_estimation, clim_estimation]
clim = [1e16, 2e17]
# clim = [0, 5]

x_axis_label = r'$\mathit{z\enspace (m)}$'
y_axis_label = r'$\mathit{r\enspace (m)}$'
# cbar_axis_label = r'$\frac{V}{m}$'
cbar_axis_label = r'$\mathit{m^{-3}}$'

plot_name = r"$\mathbf{{Mesh\enspace Color\enspace Map\enspace for\enspace {}}}$".format(probe_path)

In [ ]:
# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=7,
                   fig_height=2, 
                   fig_dpi=600,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=8,

                   x_ticklabel_start=size[1] * z_scale, 
                   y_ticklabel_start=size[0] * r_scale,
                   x_ticklabel_end=size[3] * z_scale, 
                   y_ticklabel_end=size[2] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect=aspect, image_interpolation=image_interpolation,
                   guess_number_ticks=guess_number_ticks)

o_left=0.105
o_right=0.900
o_top=0.900
o_bot=0.130

import matplotlib.pyplot as plt

plt.subplots_adjust(left=o_left,
                    bottom=o_bot,
                    right=o_right,
                    top=o_top,
                    wspace=0.010,
                    hspace=0.010)

In [ ]:
# get data
data = []

for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if ( (probe.shape == 'rec') 
        and (probe.size[0] == size[0]) 
        and (probe.size[1] == size[1]) 
        and (probe.size[2] == size[2]) 
        and (probe.size[3] == size[3]) ):
        if probe.component == cmp and (specie == None or probe.specie == specie): 
            data = reader.rec("{}".format(probe_path), size, frame)

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in reader.meta.probes:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if ( (probe.shape == 'rec') 
            and (probe.size[0] <= size[0]) 
            and (probe.size[1] <= size[1]) 
            and (probe.size[2] >= size[2]) 
            and (probe.size[3] >= size[3]) ):
            if probe.component == cmp and (specie == None or probe.specie == specie):
                data = reader.rec("{}".format(probe_path), probe.size, frame)[size[0]:size[2], size[1]:size[3]]

In [ ]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
data[128] = data[127]
data[256] = data[255]
# data[384] = data[383]
data[:,128] = data[:,127]
data[:,256] = data[:,255]
data[:,384] = data[:,383]
data[:,512] = data[:,511]
data[:,640] = data[:,639]
data[:,768] = data[:,767]
data[:,896] = data[:,895]
data[:,1024] = data[:,1023]
data[:,1152] = data[:,1151]
# data[:,1280] = data[:,1279]
# data[:,1408] = data[:,1407]
# data[:,1536] = data[:,1535]
# data[:,1664] = data[:,1663]
# data[:,1792] = data[:,1791]
# data[:,1920] = data[:,1919]
# print(len(data))

# add timestamp to plot
# plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)
plot.get_figure().suptitle('$\mathbf{t\enspace =\enspace ' + latex_float(timestamp) + 's}$', x=.85, y=.95)

# add images
plot.add_image(plot_name, data, cmap=cmap, clim=clim)

# add colorbars
plot.add_colorbar(plot_name, ticks=clim, title=cbar_axis_label)

# plot.show()

plot.save("mesh_{}_{}-{}_{}-{}_{}.png".format(
    re.sub(delimiter, '-', probe_path), size[0], size[2], size[1], size[3], timestamp))